# Amazon Recommendation System - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and build a recommendation system for a real world dataset! For this lab, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

In this lab you will: 

- Use graph-based similarity metrics to create a collaborative filtering recommender system

## Load the Dataset

In [2]:
import pandas as pd
import networkx as nx
G = nx.Graph()

df = pd.read_csv('books_data.edgelist', names=['source', 'target', 'weight'], delimiter=' ')
df.head()

,source,target,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the Metadata 

Import the metadata available in the file `'books_meta.txt'` (note it is `'\t'` seperated). 

In [3]:
# Your code here
meta = pd.read_csv('books_meta.txt', sep='\t')
meta.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [7]:
# Your code here
dresden = meta[meta.Title.str.contains('Dresden')]
dresden

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
29100,40822,0451457811,"Storm Front (The Dresden Files, Book 1)",general subject seri fantasi book scienc conte...,Book,4838,91,4.5,6,0.00
44416,62175,1571201920,Dresden Flower Garden: A New Twist on Two Quil...,general hobbi subject craft home quilt garden ...,Book,436739,0,0.0,2,0.00
70181,97857,156477435X,Dresden Fan: Quilts and Coordinating Projects ...,general hobbi subject craft home quilt garden ...,Book,522780,0,0.0,0,0.00
117904,164303,354067182X,Information Hiding: 3rd International Workshop...,general subject internet artifici comput intel...,Book,1631577,0,0.0,0,0.00
147600,205860,1859734413,Dresden : A City Reborn,general subject germani western europ book his...,Book,936790,0,0.0,0,0.00
190627,265701,0451458125,"Fool Moon (The Dresden Files, Book 2)",general subject seri fantasi book scienc conte...,Book,6710,42,4.5,1,0.00
246654,342118,1574327739,Dresden Plates of Distinction (Love to Quilt),general hobbi subject craft home quilt garden ...,Book,410291,0,0.0,4,0.60
251156,348187,0922705178,Dresden Plate Quilt: A Simplified Method,NaN,Book,1085926,0,0.0,0,0.00
284870,394126,0451458923,"Summer Knight (The Dresden Files, Book 4)",general subject fantasi book scienc fiction,Book,2259,37,4.5,1,0.00
353028,486774,0918881374,Dresden in the Ages of Splendor and Enlightenm...,art general subject paint photographi book mus...,Book,1251498,0,0.0,0,0.00


## Generate Recommendations for a Few Books of Choice

The `'books_data.edgelist'` has conveniently already calculated the distance between items for you. Given this preprocessed data, it's time to employ collaborative filtering to generate recommendations! Generate the top 10 recommendations for each book in the subset you chose. Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended. 

In [9]:
# Your code here
# Well, got a couple or extraneous results in there, but perhaps good measure for comparion.
# What does our recommender return for these books?
rec_dict = {}
id_name_dict = dict(zip(meta.ASIN, meta.Title))
for row in dresden.index:
    book_id = dresden.ASIN[row]
    book_name = id_name_dict[book_id]
    most_similar = df[(df.source == book_id)
                      | (df.target == book_id)
                     ].sort_values(by='weight', ascending=False).head(10)
    most_similar['source_name'] = most_similar['source'].map(id_name_dict)
    most_similar['target_name'] = most_similar['target'].map(id_name_dict)
    recommendations = []
    for row in most_similar.index:
        if most_similar.source[row] == book_id:
            recommendations.append((most_similar.target_name[row], most_similar.weight[row]))
        else:
            recommendations.append((most_similar.source_name[row], most_similar.weight[row]))
    rec_dict[book_name] = recommendations
    print('Recommendations for:', book_name)
    for r in recommendations:
        print(r)
    print('\n\n')

Recommendations for: Storm Front (The Dresden Files, Book 1)
('Grave Peril (The Dresden Files, Book 3)', 1.0)
('Death Masks (The Dresden Files, Book 5)', 1.0)
('Summer Knight (The Dresden Files, Book 4)', 0.75)
('More Than Mortal', 0.38)
('Never Burn a Witch: A Rowan Gant Investigation', 0.31)
('More Than Mortal (Renquist Quartet)', 0.25)



Recommendations for: Dresden Flower Garden: A New Twist on Two Quilt Classics
('Quilt Style', 1.0)
('Tradition With a Twist: Variations on Your Favorite Quilts', 1.0)



Recommendations for: Dresden Fan: Quilts and Coordinating Projects for Your Home



Recommendations for: Information Hiding: 3rd International Workshop, Ih'99 Dresden, Germany, September 29-Octer 1, 1999 Proceedings (Lecture Notes in Computer Science)



Recommendations for: Dresden : A City Reborn



Recommendations for: Fool Moon (The Dresden Files, Book 2)
('Never Burn a Witch: A Rowan Gant Investigation', 0.31)



Recommendations for: Dresden Plates of Distinction (Love to Quil

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!